---
title: "Good Simulation Code I: Starting with Functions"
subtitle: "Starting with a Monolithic Design for Small Simulations"
author: Vladislav Morozov  
format:
  revealjs:
    include-in-header: 
      text: |
        <meta name="description" content="How to write good Monte Carlo simulation code in data science: basic monolithic function design for starting, with OLS bias example (lecture note slides)"/> 
    width: 1150
    slide-number: true
    sc-sb-title: true
    incremental: true   
    logo: ../../themes/favicon.ico
    footer: "Code Design I: Using Functions"
    footer-logo-link: "https://vladislav-morozov.github.io/simulations-course/"
    theme: ../../themes/slides_theme.scss
    toc: TRUE
    toc-depth: 2
    toc-title: Contents
    transition: convex
    transition-speed: fast
slide-level: 4
title-slide-attributes:
    data-background-color: "#3c165cff"
    data-footer: " "
filters:
  - reveal-header 
embed-resources: true
include-in-header: ../../themes/mathjax.html 
highlight-style: tango
open-graph:
    description: "How to write good Monte Carlo simulation code in data science: basic monolithic function design for starting, with OLS bias example (lecture note slides)" 
---







## Introduction {background="#00100F"}
 
  

### Lecture Info {background="#43464B"}

#### Learning Outcomes

This lecture is about a starting approach to writing clean Monte Carlo code

<br>

By the end, you should be able to

- Specify a basic simulation design
- Start with a simple function-based simulation 
- Understand the importance of setting an RNG seed
- Recognize the limitations of a monolithic design


#### References

::: {.nonincremental}

Statistics:

- Chapter 4 of @Hansen2022Econometrics for OLS
- Chapter 14 of @Hansen2022Econometrics for time series basics

Programming

- Chapter 16-18 in @Lutz2025LearningPythonPowerful on functions in Python
- Chapter 2-3 of @Hillard2020PracticesPythonPro on basics of design

:::

 

### Example Of This Block {background="#43464B"}


#### Context: Bias of OLS Estimator

Consider the simple causal linear model:

$$
Y_{t}^x = \beta_0 + \beta_1 x + U_t, \quad t=1, \dots, T
$$
$Y_t^x$ — potential outcome under $x$ for observation $t$

<br>

Basic econometrics tells us that OLS estimator in regressing $Y_t$ on $(1, X_t)$ is

- Unbiased if $\E[U_t|X_1, \dots, X_T] = 0$
- Possibly biased otherwise 

#### Question of the Block: Does Bias Matter?

<div class="left-color">

Why should a user of OLS care about this bias?  
 
</div>


 

 
If bias is big, can lead to

- Improperly centered confidence intervals
- Wrong sign of point estimate
 

$\Rightarrow$ both can lead to incorrect policy conclusions if bias big


<br>

<div class="rounded-box">

Statistical question of this block: <span class="highlight">how big is the bias?</span>

</div>


#### Answer Approaches

Remember: three ways to answer statistical questions:

- Theory
- Simulations
- Empirical

. . .

<br>

This case: theory and empirical evaluation do not work

<div class="left-color">

Need to do simulations 

</div>

#### Why Not Theory?

Theory not fully satisfactory: expression for bias depends on the DGP for $(X_t, U_t)$:
$$
\E[\hat{\bbeta}] - \bbeta= \E\left[ (\bW'\bW)^{-1}\bW'\bU \right]
$$
where $\bW$ — $T\times 2$ with $t$th row given by $(1, X_t)$


- @Bao2007SecondOrderBiasMean: asymptotic order $O(1/T)$
- Unclear if actual magnitude "important in practice"


#### Why Not Use Real Data?

 
Empirical data validation not an option both in causal and predictive settings:

- Can never measure true bias even if believe in linear model
- In predictive settings: don't even care about $\hat{\bbeta}$, only about predicted $\hat{Y}_t$

. . .

<br>


<div class="left-color">

Have to recur to simulations

</div>


 
#### Questions of the Block: Simulations

::: {.nonincremental}
 
<div class="rounded-box">

Technical question of the block: <span class="highlight">how to think about and structure simulations in general?</span>
  
 
</div>

:::



<br> 

This course block — talk mostly about <span class="highlight">code design</span>

- Our three-step simulation anatomy
- Design sketches: monolithic (today) and more extensible (next lecture). Then organization and orchestration 


In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from typing import Tuple

In [ ]:
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Arial"]

BG_COLOR = "whitesmoke"
THEME_COLOR = "#3c165c"

## Specifying Our Simulation {background="#00100F"}
 

 

### Choosing Context {background="#43464B"}

#### Simulation Requirements


 

Recall that simulations should be 

- Realistic: need to think about relevant real world scenario to emulate

- Targeted: 
    - Be specific in terms of target metrics
    - Focus DGPs that cause differences in target metrics
 
<div class="left-color">

Remember: simulations necessarily limited in scope — can only do finite number of experiments $\Rightarrow$ need to focus

</div>

#### Choosing Scenario: Time Series

As an example, we care about <span class="highlight">time series</span>

<br>

What's relevant?

- Possibility of dependence across time
- Different strength of dependence
- Different lengths of time series
- (More advanced): data drift/time-varying DGP

. . . 

We'll include the first three

#### Metrics

For simplicity, we will consider just one explicit metric:

<div class="rounded-box">

Our metric: absolute bias of OLS estimator of $\beta_1$:

$$
\text{Bias}(\hat{\beta}_1) = \E[\hat{\beta}_1] - \beta_1
$$

</div>

 


Other metrics:

- Coverage of CIs based on the OLS estimator
- Proportion of incorrect signs ($\mathrm{sgn}(\hat{\beta}_1)\neq \mathrm{sgn}(\beta_1)$)



### Specifying DGPs {background="#43464B"}

#### The Problem of DGP Choice

We now have to choose data generating processes that reflect

- Dynamics of different strength
- Different sample sizes

. . .

<div class="left-color">


Ideally: would specify based on some reference empirical datasets


</div>


Here: talk in a very stylized manner 

#### Static vs. Dynamic

First dimension: include both static   and dynamic cases



:::: {.columns}

::: {.column width="46%"}

 
**Static**:

$$  \small
Y_{t} = \beta_0 + 0.5 X_{t} + U_{t}
$$

Covariate $X_t$ independent from $U_t$ and over time

:::

  
::: {.column width="8%"} 

:::

::: {.column width="46%"} 
  
**Dynamic**:

$$ \small
Y_{t} = \beta_0 + \beta_1 Y_{t-1} + U_{t}
$$

Dependence: $\beta_1$ value

$$  \small
\beta_1 \in \curl{0, 0.5, 0.95}
$$

:::

::::


<div class="left-color">

Bias appears in dynamic setting, but not static. Checking dynamic vs. static is <span class="highlight">targeted</span>

</div>


#### Sample Size and Distributions of $U_t, X_t, Y_0$

<div class="left-color">


::: {.nonincremental}

Second and third design dimension:

2. Sample sizes
3. DGPs for variables not determined by model

:::

</div>


To start with — keep things simple to focus on essentials:

- One DGP per $U_t, X_t, Y_0$: each $N(0, 1)$
- $T=50, 200$ 


. . .

Different sample sizes: how bias changes (targeted)






## Basic Functional Implementation {background="#00100F"}
 

 

### Implementation Approaches {background="#43464B"}


#### Summary So Far

So far specified:

- Metric
- How each dataset should be generate
- Size of each dataset

. . .

<br>

<div class="left-color">

Now time to actually implement — but how?

</div>

#### Two Approaches

In this block talk about two approaches: 

- <span class="highlight">Today</span>: monolithic, based on a single function (good for starting and small simulations)
- <span class="highlight">Next time</span>: a more modular approach (better for larger simulations and easier to develop)

<br>

<div class="left-color">

Code examples of today: illustrative examples

</div>


#### General Advice

How to not get overwhelmed — a good universal rule: 

<div class="rounded-box">

Start with the simplest pieces and iterate from there


</div>
 
- Simulations are often fairly complex piece of code
- Do not try to write the whole thing in one go
- Do not be afraid of rewriting and improving things later


. . . 

<br>

Often: start with basic simulation function (today) and then refactor and split up (next time)


::: {.footer}

A good way to work is to follow a lightweight agile approach

:::


### Basic Function {background="#43464B"}



#### Practice: Starting with the Static Case
 

Our simplest thing: static model with a small sample
$$
Y_{t} = \beta_0 + \beta_1 X_t + U_t, \quad T=50
$$



Remember simulation anatomy: `for` many datasets we

1. Draw 50 points $(X_t, Y_t)$
2. Run OLS estimator $\hat{\beta}_1$
3. Compute error $\hat{\beta}_1-\beta_1$

Simplest: wrap in a single function

#### Example: Basic Simulation Function {.scrollable}


In [ ]:
#| echo: true
def simulate_ols(
    n_sim: int = 1000, n_obs: int = 50, beta0: float = 0.0, beta1: float = 0.5
) -> list[float]:
    """Simulates OLS estimation for a static/exogenous DGP: y = beta0 + beta1*x + u.

    Args:
        n_sim (int): Number of simulations to run. Defaults to 1000.
        n_obs (int): Number of observations per simulation. Defaults to 100.
        beta0 (float): True intercept value. Defaults to 0.
        beta1 (float): True slope coefficient. Defaults to 0.5.

    Returns:
        list[float]: List of errors of beta1 estimates for each simulation.
    """

    # Create container to store results
    results_ols_errors = []

    # Initialize the RNG
    rng = np.random.default_rng()

    # Core simulation loop
    for _ in range(n_sim):
        # 1. Draw data
        x = rng.normal(size=n_obs)
        u = rng.normal(size=n_obs)
        y = beta0 + beta1 * x + u

        # 2. Run OLS estimation
        W = np.column_stack([np.ones(n_obs), x])
        beta_hat = np.linalg.inv(W.T @ W) @ W.T @ y

        # 3. Compute error
        results_ols_errors.append(beta_hat[1] - beta1)

    return results_ols_errors

#### Our First Results: Static Model

Now can execute our simulation by calling function with default arguments:

In [ ]:
#| echo: true
static_results = simulate_ols()
print(f"Bias (static DGP): {np.mean(static_results)}")

Qualitatively:

- Bias small relative to coefficients (recall $\beta_1=0.5$)
- Theory is confirmed

::: {.footer}

In practice you would usually wrap the function in a script and call from the command line

:::

#### Distribution of MC Estimates

Can also take a look at density of estimates:


In [ ]:
fig, ax = plt.subplots(figsize=(14, 4.5))
fig.patch.set_facecolor(BG_COLOR)
fig.patch.set_edgecolor(THEME_COLOR)
fig.patch.set_linewidth(5)
sns.kdeplot(static_results, ax=ax, color=THEME_COLOR)
plt.axvline(0, color=THEME_COLOR, linestyle="--", label="No bias")
plt.axvline(np.mean(static_results), color="brown", linestyle="--", label="MC Estimate")
plt.title(
    f"Distribution of OLS Estimation Error under Static Model",
    loc="left",
)
plt.xlabel("Estimation error")
plt.legend()
plt.show()

### Internal Reproducibility: Setting Seeds {background="#43464B"}


#### Problem: Lack of Reproducibility

What happens if we rerun our code twice?


In [ ]:
#| echo: true
static_results_1 = simulate_ols()
static_results_2 = simulate_ols()
print(f"Bias (first run): {np.mean(static_results_1)}")
print(f"Bias (second run): {np.mean(static_results_2)}")

. . .

- Same qualitative result
- <span class="highlight">Different</span> numerical results

. . .

<div class="left-color">

Our results are not <span class="highlight">reproducible</span> 

</div>

#### Why No Reproducibility and How to Fix?

Different data drawn by `rng` in different runs of the function

. . . 

<br>

To get same data, need to know that:

- Computers cannot generate actual random numbers ([at least quickly and with guaranteed distribution](https://engineering.mit.edu/engage/ask-an-engineer/can-a-computer-generate-a-truly-random-number/))
- Instead used <span class="highlight">pseudo</span>random number generators — deterministic algorithms with "good" properties
- PseudoRNG controlled by <span class="highlight">seeds</span>

<div class="left-color">

Give same seed = get the same sequence of numbers

</div>

::: {.footer}

For example, see [Wikipedia](https://en.wikipedia.org/wiki/Pseudorandom_number_generator) on pseudoRNG

:::

#### Setting the RNG Seed

In practice: easy to provide a `seed` to our `rng`


In [ ]:
#| echo: true
#| code-line-numbers: "1,6,15,25"
def simulate_ols(
    n_sim: int = 1000,
    n_obs: int = 50,
    beta0: float = 0.0,
    beta1: float = 0.5,
    seed: int = 1,
) -> list[float]:
    """Simulates OLS estimation for a static/exogenous DGP: y = beta0 + beta1*x + u.

    Args:
        n_sim (int): Number of simulations to run. Defaults to 1000.
        n_obs (int): Number of observations per simulation. Defaults to 100.
        beta0 (float): True intercept value. Defaults to 0.
        beta1 (float): True slope coefficient. Defaults to 0.5.
        seed (int): random number generator seed. Defaults to 1.

    Returns:
        list[float]: List of errors of beta1 estimates for each simulation.
    """

    # Create container to store results
    results_ols_errors = []

    # Initialize the RNG
    rng = np.random.default_rng(seed)

    # Core simulation loop
    for _ in range(n_sim):
        # 1. Draw data
        x = rng.normal(size=n_obs)
        u = rng.normal(size=n_obs)
        y = beta0 + beta1 * x + u

        # 2. Run OLS estimation
        W = np.column_stack([np.ones(n_obs), x])
        beta_hat = np.linalg.inv(W.T @ W) @ W.T @ y

        # 3. Compute error
        results_ols_errors.append(beta_hat[1] - beta1)

    return results_ols_errors

::: {.footer}

Setting a seed guarantees the same sequence of (pseudo)random numbers

:::
 
#### Rerunning Simulations
 
Now rerunning with the same (default) seed gives the same results:


In [ ]:
#| echo: true
static_results_1 = simulate_ols()
static_results_2 = simulate_ols()
print(f"Bias (first run): {np.mean(static_results_1)}")
print(f"Bias (second run): {np.mean(static_results_2)}")

. . .

<br>

 
<div class="rounded-box">

<span class="highlight">Always set seeds explicitly to help reproducibility</span>

</div>

## Expanding the Simulation {background="#00100F"}
 
### Adding a Dynamic DGP {background="#43464B"}


#### Expanding the Simulation
 
<div class='left-color'>

Now need to expand simulations to add AR(1) design

</div>
 
Simplest way: just expand existing function

- Add new DGP option `"ar1"`
- Involves changing `simulate_ols` to have a new argument for `dgp_type`
  - Old behavior: `dgp_type="static"`
  - New behavior: `dgp_type="ar1"`
- Also deal with losing one observation to $Y_{t-1}$

#### Expanded Simulation Function: With AR(1) DGP


In [ ]:
#| echo: true
#| code-line-numbers: "7,16,35-43"
def simulate_ols(
    n_sim: int = 1000,
    n_obs: int = 50,
    beta0: float = 0.0,
    beta1: float = 0.5,
    seed: int = 1,
    dgp_type: str = "static",
) -> list[float]:
    """Simulates OLS estimation for static or AR(1) DGP.
    Args:
        n_sim (int): Number of simulations to run. Defaults to 1000.
        n_obs (int): Number of observations per simulation. Defaults to 100.
        beta0 (float): True intercept value. Defaults to 0.
        beta1 (float): True slope coefficient. Defaults to 0.5
        seed (int): random number generator seed. Defaults to 1.
        dgp_type (str): Type of DGP: "static" or "ar1". Defaults to "static".

    Returns:
        list[float]: List of errors of beta1 estimates for each simulation.
    """

    # Create container to store results
    results_ols_errors = []

    # InitializeRNG
    rng = np.random.default_rng(seed)

    # Core simulation loop
    for _ in range(n_sim):
        # 1. Draw data
        if dgp_type == "static":
            x = rng.normal(size=n_obs)
            u = rng.normal(size=n_obs)
            y = beta0 + beta1 * x + u
        elif dgp_type == "ar1":
            y = np.zeros(n_obs + 1)
            u = rng.normal(size=n_obs + 1)
            y[0] = beta0 + u[0]
            for t in range(1, n_obs + 1):
                y[t] = beta0 + beta1 * y[t - 1] + u[t]
            # Use the last n_obs elements of y as and the first n_obs as x
            x = y[:-1]  # x is y lagged (n_obs elements)
            y = y[1:]  # y[1:] is the dependent variable (n_obs elements)
        else:
            # Handle the case of giving the wrong DGP value
            raise ValueError("Invalid DGP choice")

        # 2. Run OLS estimation
        W = np.column_stack([np.ones(n_obs), x])
        beta_hat = np.linalg.inv(W.T @ W) @ W.T @ y

        # 3. Compute error
        results_ols_errors.append(beta_hat[1] - beta1)

    return results_ols_errors

#### Running AR(1) Simulation
 
Can now run our simulations with $\beta \in \curl{0, 0.5, 0.95}$:


In [ ]:
#| echo: true
#| code-line-numbers: "1-3"
ar_results_no_pers = simulate_ols(beta1=0, dgp_type="ar1")
ar_results_med_pers = simulate_ols(beta1=0.5, dgp_type="ar1")
ar_results_high_pers = simulate_ols(beta1=0.95, dgp_type="ar1")
print(f"Bias (no persistence): {np.mean(ar_results_no_pers):.3f}")
print(f"Bias (medium persistence): {np.mean(ar_results_med_pers):.3f}")
print(f"Bias (high persistence): {np.mean(ar_results_high_pers):.3f}")

. . .

Conclusions:

<div class='left-color'>

- More persistence = larger absolute bias
- Direction: downward (underestimating)

</div>
  
#### MC Distribution Under Persistence


In [ ]:
fig, ax = plt.subplots(figsize=(14, 4.5))
fig.patch.set_facecolor(BG_COLOR)
fig.patch.set_edgecolor(THEME_COLOR)
fig.patch.set_linewidth(5)
sns.kdeplot(ar_results_high_pers, ax=ax, color=THEME_COLOR)
plt.axvline(0, color=THEME_COLOR, linestyle="--", label="No bias")
plt.axvline(np.mean(ar_results_high_pers), color="brown", linestyle="--", label="MC Estimate")
plt.title(
    f"Distribution of OLS Estimation Error under AR(1) with $\\beta_1=0.95$",
    loc="left",
)
plt.xlabel("Estimation error")
plt.legend()
plt.show()

#### Effect of Sample Size
 
To check effect of sample size, use different values for `n_obs`:


In [ ]:
#| echo: true
#| code-line-numbers: "1-3"
ar_results_no_pers = simulate_ols(beta1=0, n_obs=200, dgp_type="ar1")
ar_results_med_pers = simulate_ols(beta1=0.5, n_obs=200, dgp_type="ar1")
ar_results_high_pers = simulate_ols(beta1=0.95, n_obs=200, dgp_type="ar1")
print(f"Bias (no persistence): {np.mean(ar_results_no_pers):.3f}")
print(f"Bias (medium persistence): {np.mean(ar_results_med_pers):.3f}")
print(f"Bias (high persistence): {np.mean(ar_results_high_pers):.3f}")

. . .

Conclusions:

<div class='left-color'>

Bias decays as sample size increases

</div>
  

### Limitations of Functional Approach {background="#43464B"}

#### What's Good About The Functional Approach



The monolithic function approach has a few pluses in our <span class="highlight">small</span> simulation:

- Was easy to create and design: just put all the components in one function in the same order
- Everything is in one place: easy to look at the whole code

. . . 

<br>

<div class='left-color'>

These advantages $\Rightarrow$ why writing a simple function is usually a good starting place for new simulations


</div>

#### Problem: How To Expand Simulations?

By now: a working base simulation

. . . 

<br>


But what if we <span class="highlight">expand our simulations</span>?

- Try more DGPs (e.g. heavier tails for innovations, different $X$)
- More sample sizes
- Different models (not necessarily $Y_t = \beta_0 + \beta_1 X_t + U_t$)


#### Limitations of Current Approach

<div class='left-color'>

We need a better approach to organizing simulations

</div>

- Basic approach is <span class="highlight">fine</span> if you just want run one thing
- But difficult if you want to expand more:
    - Adding new DGPs would mean expanding simulation function even more
    - We would have to remember to run all the different cases ourselves
 
. . .

Becomes difficult to scale and maintain

::: {.footer}

The simulation function becomes a ["god object"](https://en.wikipedia.org/wiki/God_object)

:::


## Recap and Conclusions {background="#00100F"}
 
#### Recap

<br>

In this lecture we 
 
- Saw our first simulation
- Talked about structuring simulation code in a single function + advantages and limitations of this approach
- Discussed setting RNG seeds for reproducibility
  

#### Next Questions

<br>

- How to use a more modular and loosely modular design for larger simulations?
- How to run many simulation scenarios at the same time?
- How to apply these approaches to different statistical scenarios?

#### References {.allowframebreaks visibility="uncounted"}

::: {#refs}
:::